In [120]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [5]:
df = pd.read_csv("./pm10_final_v4.csv", index_col = 0)

In [6]:
df.head()

,"('PM10', '강원 강릉시')","('PM10', '광주 북구')","('PM10', '대전 중구')","('PM10', '부산 해운대구')","('PM10', '서울 서대문구')","('강수량', '90')","('강수량', '101')","('강수량', '102')","('강수량', '105')","('강수량', '108')",...,Passenger Traffic of Highways Current Period(10000 persons),Passenger Traffic of Waterways Current Period(10000 persons),Freight Traffic Current Period(10000 tons),Freight Traffic of Railways Current Period(10000 tons),Freight Traffic of Highways Current Period(10000 tons),Freight Traffic of Waterways Current Period(10000 tons),Manufacturing Purchasing Managers'' Index(%),cn_a50,cn_szse,cn_sha
2015-01-01 03:00:00,35.0,21.7,33.3,18.7,63.7,0.0,0.0,0.0,0.0,0.0,...,150028.0,1483.0,364076.0,31253.0,283653.0,49119.0,49.8,11860.36,11520.59,3350.52
2015-01-01 06:00:00,45.3,21.0,61.3,19.0,113.3,0.0,0.0,0.0,0.0,0.0,...,150028.0,1483.0,364076.0,31253.0,283653.0,49119.0,49.8,11860.36,11520.59,3350.52
2015-01-01 09:00:00,56.3,21.7,95.0,28.3,135.0,0.0,0.0,0.0,0.0,0.0,...,150028.0,1483.0,364076.0,31253.0,283653.0,49119.0,49.8,11860.36,11520.59,3350.52
2015-01-01 12:00:00,68.7,41.0,124.0,53.5,85.0,0.0,0.0,0.0,0.0,0.0,...,150028.0,1483.0,364076.0,31253.0,283653.0,49119.0,49.8,11860.36,11520.59,3350.52
2015-01-01 15:00:00,88.3,70.3,56.3,54.3,44.7,0.0,0.0,0.0,0.0,0.0,...,150028.0,1483.0,364076.0,31253.0,283653.0,49119.0,49.8,11860.36,11520.59,3350.52


In [17]:
df = df.dropna()

### 랜덤포레스트 회귀 모델로 중요 피처 확인하기

In [18]:
x = df.drop(["('PM10', '강원 강릉시')", "('PM10', '광주 북구')", "('PM10', '대전 중구')", 
             "('PM10', '부산 해운대구')", "('PM10', '서울 서대문구')"], axis = 1)
# 서울의 서대문구를 종속변수로 설정
y = df["('PM10', '서울 서대문구')"]

In [102]:
rf = RandomForestRegressor(n_jobs = -1, random_state = 10)

In [103]:
rf.fit(x, y)

RandomForestRegressor(n_jobs=-1, random_state=10)

In [104]:
feat_imp = rf.feature_importances_

In [149]:
sum(pd.Series(feat_imp).sort_values(ascending = False)[0:200])

0.7678465157998684

In [162]:
feat100_1 = x.iloc[:, pd.Series(feat_imp).sort_values(ascending = False)[0:100].index].columns

- 영향력이 큰 100개의 지표 (**RandomForest** 기준)

- 국내 사회경제 지표는 8개가 해당됨
    - 코스피, 코스닥 모두 포함됨
    - 석탄소비량과 고급휘발유 등의 화석연료 지표가 포함되어 있음
    - 상관계수는 음의 상관관계를 가짐 (절댓값이 0.3 이하)
- 중국 사회경제 지표는 7개가 해당됨
    - 순위가 약간 낮긴하지만 상해종합(cn_sha)과 szse이 포함됨
    - 상관계수는 음의 상관관계를 가짐 (절댓값이 0.3 이하)

### 그래디언트부스팅 시도

In [121]:
model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(x, y)

GradientBoostingRegressor(random_state=42)

In [156]:
feat_imp2 = model.feature_importances_

In [163]:
feat100_2 = x.iloc[:5, pd.Series(feat_imp2).sort_values(ascending = False)[0:100].index].columns

- 영향력이 큰 50개의 지표 (**GradientBoosting** 기준)

- 국내 사회경제 지표는 8개
- 중국 사회경제 지표는 6개
    - 특히 산업 생산량을 중요시 여기는 것이 파악됨

#### 공통부문 정리

- 전라도의 운수창고업은 상위권에 2회 등장하며 상관관계에서도 -0.429로 상관관계가 높은 것으로 파악됨
    - 전라도의 항만은 주로 중국과의 무역을 할 것으로 추측됨
    - 목포항, 군산항, 여수항 3개의 무역항이 있으며 중국과의 관련성은 현재 탐색 중
    - 군산항 https://encykorea.aks.ac.kr/Article/E0006660, https://www.hankyung.com/article/202401298975Y
    
- 강원도의 광업 생산량 역시 상위권에 2회 등장한다. 상관관계는 0.163으로 높지는 않음
    - 강원도 광업 https://www.kigam.re.kr/menu.es?mid=a30103010204
    - 생산량의 대부분이 석회석으로 광산에서 미세먼지가 배출이되며 석회석은 주로 건설용도로 사용되는데 건설 현장에서도 미세먼지가 발생함
    - 석회석 광산과 미세먼지 http://www.ombudsmannews.com/default/index_view_page.php?part_idx=4847&idx=9560
    - 건설현장과 미세먼지 https://www.jekosae.or.kr/xml/02729/02729.pdf

## 경상도 관련 분석

In [150]:
y2 = df["('PM10', '부산 해운대구')"]

### 랜덤포레스트 시도 (부산)

In [151]:
rf2 = RandomForestRegressor(n_jobs = -1, random_state = 10)

In [152]:
rf2.fit(x, y2)

RandomForestRegressor(n_jobs=-1, random_state=10)

In [158]:
feat_imp3 = rf2.feature_importances_

In [164]:
feat100_3 = x.iloc[:, pd.Series(feat_imp2).sort_values(ascending = False)[0:100].index].columns

### 그레디언트 부스팅 시도 (부산)

In [155]:
model2 = GradientBoostingRegressor(n_estimators=100, random_state=42)
model2.fit(x, y2)

GradientBoostingRegressor(random_state=42)

In [160]:
feat_imp4 = model2.feature_importances_

In [165]:
feat100_4 = x.iloc[:, pd.Series(feat_imp4).sort_values(ascending = False)[0:100].index].columns

### 상위 100개의 피처 4개 중 여러번 등장한 피처들 리스트 뽑아보기

In [175]:
feat_concat = pd.concat([pd.Series(feat100_1), pd.Series(feat100_2), pd.Series(feat100_3), pd.Series(feat100_4)]).drop_duplicates()

In [178]:
feat_concat = pd.DataFrame(feat_concat)

In [180]:
feat_concat.columns = ["피처명"]

In [183]:
feat_concat = feat_concat.set_index("피처명")

In [186]:
feat_concat.loc[:, "count"] = 0

In [188]:
for feat in feat_concat.index:
    count = 0
    if feat in feat100_1:
        count += 1
        
    if feat in feat100_2:
        count += 1
        
    if feat in feat100_3:
        count += 1
        
    if feat in feat100_4:
        count += 1
        
    feat_concat.loc[feat, "count"] = count

In [195]:
feat_concat = feat_concat.sort_values(by = "count", ascending = False)

In [199]:
feat_concat[feat_concat["count"] == 4]

,count
피처명,
"('GUST풍속', '22104')",4
crude_steel,4
"('기온', '22106')",4
"('전라도', '정보통신업')",4
insulating_glass,4
"('해면기압', '57494.0')",4
"('이슬점 온도', '50745.0')",4
"('기온', '22102')",4
"('이슬점 온도', '59287.0')",4


In [202]:
feat_concat[feat_concat["count"] == 3]

,count
피처명,
"('GUST풍속', '22106')",3
"('해면기압', '58238.0')",3
"('현지기압', '56029.0')",3
tires,3
"('이슬점 온도', '58847.0')",3
"('기온', '22103')",3
"('습도', '58666.0')",3
"('이슬점온도', '102')",3
"('기온', '90')",3


- 33개의 피처가 4번 모두 등장
- 32개의 피처가 3번 등장

- 중국의 단열 유리 생산은 중요도가 상당히 높은 피처로 파악됨
    - 상관계수 분석에서도 일반 유리, 접합 유리, 단열 유리 모두 유의미한 것으로 파악됨
    - 유리 생산과 먼지 오염 관련 내용 https://www.agc-glass.eu/en/sustainability/decarbonisation/environmental-impact

- 타이어 역시 3번 등장한 중요도가 높은 피처
    - 도로 주행 시 타이어 마모로 인한 미세먼지 발생 https://news.kbs.co.kr/news/pc/view/view.do?ncd=5575028

- 철강 관련 제품들의 생산량이 중요도가 높게 책정됨
    - 제철소의 미세먼지 발생 관련 기사 https://www.idjnews.kr/news/articleView.html?idxno=137144

- 보통 휘발유와 고급 휘발유 모두 4회, 3회 포함됨
    - 자동차와 미세먼지 발생 관련 기사 https://www.pressian.com/pages/articles/2021030508311008716